In [1]:
import os
import cv2
import torchvision
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary

In [2]:
test_data_PATH = os.path.join(os.getcwd(), "data\\test_data.csv")
train_data_PATH = os.path.join(os.getcwd(), "data\\train_data.csv")
label_PATH = os.path.join(os.getcwd(), "data\\label.csv")

In [3]:
test_data = pd.read_csv(test_data_PATH)
test_data.head()

,img,label
0,[[[247 247 247]\n [247 247 247]\n [247 247 2...,0
1,[[[255 255 255]\n [255 255 255]\n [255 255 2...,0
2,[[[255 255 255]\n [255 255 255]\n [255 255 2...,0
3,[[[255 255 255]\n [255 255 255]\n [255 255 2...,0
4,[[[255 255 255]\n [255 255 255]\n [255 255 2...,0
